## Problem 1

# a)

We want to obtain log-likelihood function for a random sample of n observations. Our likelihood function looks like this:

\begin{equation*}
L(\alpha, \beta | x ) = \Pi_{i=1}^n f(x_i | \alpha , \beta )
\end{equation*}

Now we logarithmize both sides of likelihood function and perform several computations in order to simplify result:

\begin{equation*}
\ln L(\alpha, \beta | x ) = \sum_{i=1}^n \ln f(x_i | \alpha , \beta ) = 
\sum_{i=1}^n \ln (\alpha \beta x_{i}^{\beta-1}e^{-\alpha x_i^\beta}) = 
\sum_{i=1}^n \Big( \ln(\alpha \beta) + (\beta-1) \ln(x_i)-\alpha x_i^\beta \Big) = 
n \ln (\alpha \beta ) + (\beta - 1) \sum_{i=1}^n \Big(\ln(x_i)\Big) - \alpha \sum_{i=1}^n x_i^\beta
\end{equation*}


Our log-likelihood function for a random sample of n observations:
\begin{equation*}
\ln L(\alpha, \beta | x ) = n \ln (\alpha \beta ) + (\beta - 1) \sum_{i=1}^n (\ln(x_i)) - \alpha \sum_{i=1}^n x_i^\beta
\end{equation*}

# b)

Necessary condition equations for $\alpha$ and $\beta$ can be easily derived by taking first derivatives of the log-likelihood function with respect to $\alpha$ and $\beta$ respectively and setting them to be equal to zero:\
(This makes sense, as we are trying to maximize the log-likelihood function) 


$$
\left[\begin{array}{cc} 
\frac{\partial \ln L}{\partial \alpha} \\
\frac{\partial \ln L}{\partial \beta}
\end{array}\right] =
\left[\begin{array}{cc} 
\frac{n}{\alpha} - \sum_{i=1}^n x_i^\beta \\ 
\frac{n}{\beta} + \sum_{i=1}^n \Big( \ln ( x_i ) \Big) - \alpha \ln (\beta) \sum_{i=1}^n x_i^\beta
\end{array}\right] =
\left[\begin{array}{cc} 
0 \\ 
0
\end{array}\right]
$$


From this we can easily derive that $\alpha = \frac{n}{\sum_{i=1}^n x_i^\beta}$. In order to obtain maximizing value of $\beta$ we need to plug into second equation vlaue of $\alpha$:


\begin{equation*}
0 = \frac{n}{\beta} + \sum_{i=1}^n \Big( \ln ( x_i ) \Big) - \alpha \ln (\beta) \sum_{i=1}^n x_i^\beta = 
\frac{n}{\beta} + \sum_{i=1}^n \Big( \ln ( x_i ) \Big) - \frac{n}{\sum_{i=1}^n x_i^\beta} \ln (\beta) \sum_{i=1}^n x_i^\beta =
\frac{n}{\beta}-n \ln (\beta)
\end{equation*}

\begin{equation*}
0 = \frac{n}{\beta}-n \ln (\beta)
\end{equation*}

\begin{equation*}
\beta \ ln \beta = 1
\end{equation*}


This is hardly "nice" solution. To obtain value of $\beta$ as a real number, we need to apply Lambert W function:

\begin{equation*}
\beta \ ln \beta = 1
\end{equation*}

\begin{equation*}
e^{ \ln \beta} \ ln \beta = 1
\end{equation*}

\begin{equation*}
\ln \beta = W(1)
\end{equation*}

\begin{equation*}
\beta = e^W(1)
\end{equation*}





